### 1. 패키지 설치

In [17]:
pip install --upgrade pip

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [18]:
pip install -r requirements.txt  # 패키지 설치

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [19]:
import torch
print(torch.__version__)  # PyTorch 버전
print(torch.version.cuda)  # PyTorch가 사용하는 CUDA 버전
print(torch.cuda.is_available())  # GPU 사용 가능 여부


2.4.1+cu121
12.1
True


### 2. 문서 split 및 Chroma를 활용한 vector store 구성

In [50]:
from huggingface_hub import whoami

try:
    user_info = whoami()
    print(f"로그인 상태입니다. 사용자: {user_info['name']}")
except Exception as e:
    print("로그인되지 않았거나 토큰이 유효하지 않습니다.")
    print(e)

로그인 상태입니다. 사용자: chaeeee


In [51]:
import os
os.environ["LANGCHAIN_TRACING_V2"] = "false"

In [52]:
from langchain.schema import Document
import os, json
from tqdm import tqdm

# 1. JSON 파일 경로 설정
json_path = "./json_data.json"  # 단일 JSON 파일 경로

# 2. JSON 데이터 불러오기
with open(json_path, 'r', encoding='utf-8') as f:
    all_data = json.load(f)

# 3. {}를 기준으로 JSON 데이터 분할 및 Document 객체 생성
documents = []

for data in tqdm(all_data, desc="Generating Documents", unit="entry", ncols=80):
    metadata = {
        "title": data.get('title', 'N/A'),
        "artist": data.get('artist', 'N/A'),
        "year": data.get('year', 'N/A'),
        "read_count": data.get('read_count', 0)
    }

    # JSON 데이터의 각 항목을 Document 객체로 변환
    doc_content = json.dumps(data, ensure_ascii=False, indent=4)
    documents.append(Document(
        page_content=doc_content.strip(),
        metadata=metadata
    ))

Generating Documents: 100%|██████████| 11029/11029 [00:01<00:00, 8782.98entry/s]


In [53]:
len(documents)

11029

In [54]:
documents[1]

Document(metadata={'title': '팔괘호', 'artist': '한기석', 'year': '1960', 'read_count': 129}, page_content='{\n    "title": "팔괘호",\n    "title_ch": "八卦壺",\n    "title_eng": "Palgwae Vase",\n    "artist": "한기석",\n    "artist_eng": "HAN Kisuk",\n    "artwork_number": 2,\n    "year": "1960",\n    "size": "250×127",\n    "materials": "캔버스, 종이에 유화 물감",\n    "category": "회화 II",\n    "description": "한농(韓農) 한기석(1930-2011)은 표면 묘사에 많은 관심을 가진 작가이다.그는 모나고 약간 무게가 있는 듯이 보이는 항아리와 화병을 계속 그렸는데, 항아리에 대한 사람들의 일반적 향수 이미지는 그의 작품에서 친근감을 불어넣어 준다. 또한 완벽한 균형을 이루면서 노련하게 표현되어 다양한 색의 조화를 지닌 도자기의 미를 느낄 수 있다.",\n    "read_count": 129\n}')

In [6]:
# 특정 Document 객체의 텍스트 길이 확인
len(documents[11001].page_content)

1009

In [7]:
# 800자를 초과하는 Document 개수 세기
over_800_count = sum(1 for doc in documents if len(doc.page_content) > 800)

# 결과 출력
print(f"800자를 초과하는 Document 개수: {over_800_count}")


800자를 초과하는 Document 개수: 6460


In [55]:
from langchain.vectorstores import FAISS
from sentence_transformers import SentenceTransformer
import faiss  # FAISS 라이브러리 필요

# 1. 임베딩 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")


You try to use a model that was created with version 3.3.1, however, your version is 3.2.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [ ]:
# 2. 문서 데이터와 메타데이터 분리
texts = [doc.page_content for doc in documents]  # 문서 텍스트
metadatas = [doc.metadata for doc in documents]  # 문서 메타데이터

# 3. 문서 임베딩 생성
embeddings = embedding_model.encode(texts)

In [ ]:
# 4. FAISS 인덱스 생성
embedding_dim = embeddings.shape[1]  # 벡터 차원 확인
faiss_index = faiss.IndexFlatL2(embedding_dim)  # L2 거리 기반 인덱스
faiss_index.add(embeddings)  # 벡터 추가


In [ ]:
from langchain.docstore.in_memory import InMemoryDocstore

# 5. Docstore 생성
# 각 문서에 고유 ID를 부여해 InMemoryDocstore 생성
docstore = InMemoryDocstore({i : doc for i, doc in enumerate(documents)})

In [ ]:
# 6. FAISS 벡터스토어 생성
def embed_query(text):
    return embedding_model.encode([text])[0]  # 단일 쿼리 텍스트를 임베딩

index_to_docstore_id = {i: i for i in range(len(documents))}

faiss_db = FAISS(
    embedding_function=embed_query,
    index=faiss_index,
    docstore=docstore,
    index_to_docstore_id=index_to_docstore_id
)

# 7. FAISS 데이터베이스 저장
faiss_db.save_local("./faiss_artworks_0304_artworks")
print("FAISS 데이터베이스가 성공적으로 저장되었습니다!")


`embedding_function` is expected to be an Embeddings object, support for passing in a function will soon be removed.


FAISS 데이터베이스가 성공적으로 저장되었습니다!


In [13]:
# 9. 검색 테스트
query = "박승무의 설경"
results = faiss_db.similarity_search(query, k=5)

# 10. 검색 결과 출력
for result in results:
    print("문서 텍스트:", result.page_content)
    print("문서 메타데이터:", result.metadata)
    print('=------------------------------------------')


문서 텍스트: {
    "title": "설경",
    "title_ch": "雪景",
    "title_eng": "Snowscape",
    "artist": "박승무",
    "artist_eng": "PARK Seungmoo",
    "artwork_number": 3371,
    "year": "1974",
    "size": "65×255",
    "materials": "종이에 먹, 색",
    "category": "회화 I",
    "description": "심향(深香)박승무(1893-1980)는 설경을 위주로 작품을 해왔다. 눈 덮인 초가집과 나무들의 사실적인 표현과는 달리 배경을 이루는 산의 묘사는 구도에서 도식적이리 만큼 빈번히 등장한다. 먼 산의 윤곽선 및 나무들의 부드럽고 정감 짙은 풍정미, 잔 점과 선의 표현적 특질 등이 그의 독자적인 양식으로 간주된다.<설경>(1974)이 제작된 무렵 박승무는 설경의 명수로서 유명하였으며 종축보다는 단독 회화성이 더 짙은 횡축그림을 많이 그렸다. 그는 자연이 지니고 있는 가장 원형적인 것을 찾아내고 자연의 원형을 자기화함으로써 독특한 양식을 이루어 내었다. 이 작품은 한국의 야산을 대상으로 하여 향토주의 수묵화를 추구하고 있다. 화면 상단의 하늘과 하단의 강물을 중묵의 음영으로 처리하여 설경의 효과를 더욱 실감나게 하였다.",
    "read_count": 74
}
문서 메타데이터: {'title': '설경', 'artist': '박승무', 'year': '1974', 'read_count': 74}
=------------------------------------------
문서 텍스트: {
    "title": "설경(雪景)",
    "title_ch": "N/A",
    "title_eng": "Snowy Landscape",
    "artist": "박승무",
    "artist_eng": "PARK Seungmoo",
    "artwork_number":

In [14]:
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

You try to use a model that was created with version 3.3.1, however, your version is 3.2.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





In [1]:
from langchain_community.vectorstores import FAISS


# 기존 DB 로드 
persist_directory = "./faiss_artworks_0304"

try:
    faiss_db = FAISS.load_local(
        folder_path=persist_directory,
        embeddings=embedding_model,
        allow_dangerous_deserialization=True  # 신뢰할 수 있는 소스에서만 사용
    )
    
    # embedding_function 수정
    faiss_db.embedding_function = lambda text: (
        embedding_model.encode(text) if isinstance(text, str) else embedding_model.encode(str(text))
    )
    
    print("FAISS 데이터베이스가 성공적으로 로드되었습니다!")
except Exception as e:
    print(f"FAISS 데이터베이스 로드 중 오류 발생: {e}")

FAISS 데이터베이스 로드 중 오류 발생: name 'embedding_model' is not defined


In [15]:
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4bit 양자화 활성화
    bnb_4bit_compute_dtype=torch.float16,  # 계산 타입 설정 (float16이 일반적)
    bnb_4bit_use_double_quant=True,  # 더블 양자화 사용 (메모리 절약)
    bnb_4bit_quant_type="nf4",  # NormalFloat4 (NF4) 사용
)

In [16]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,  # 4-bit 양자화 활성화
    bnb_4bit_compute_dtype="float16",  # 계산 정밀도 설정
    bnb_4bit_quant_type="nf4",  # NF4 양자화 방식 사용 (효율적)
    bnb_4bit_use_double_quant=True,  # 이중 양자화 사용
)

In [17]:
import torch
from langchain import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline

# 모델과 토크나이저 로드 (CUDA 사용)
model_id = "deepseek-ai/DeepSeek-R1-Distill-Qwen-14B"
tokenizer = AutoTokenizer.from_pretrained(model_id, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    quantization_config=bnb_config,  # ✅ 올바른 양자화 설정 적용
    device_map="auto",  # ✅ 자동 GPU 배치
    trust_remote_code=True,
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:27<00:00,  6.90s/it]


In [18]:
from transformers import pipeline

# 파이프라인 생성
pipe = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
llm = HuggingFacePipeline(pipeline=pipe)

/tmp/ipykernel_798155/1315975019.py:15: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [19]:
from langchain.prompts import ChatPromptTemplate

deepseek_template = """
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions **accurately based on the provided document (context)**. 
If the requested information is not found in the document, respond with:
"문서에 해당 정보가 없습니다." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, **bold** or *italic* text) to enhance readability.
You must include the artwork number in your response.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
"""




exaone_template = '''
<|system|>
You are an AI assistant tasked with refining and polishing the provided logical reasoning into a final answer in Korean.  
Your role is to produce a clear, concise, and well-structured response that maintains the original meaning and key details.  
Ensure that your final answer is written in Korean and uses **beautiful Markdown formatting** (e.g., headings, bullet points, **bold** or *italic* text) to enhance readability.  
Focus solely on refining the content without adding any new information.
You must include the artwork number in your response.

<|reasoning|>
{reasoning}

<|user|>
Based on the above reasoning, please generate a refined and final answer in Korean.

<|assistant|>
'''



# DeepSeek 템플릿 생성
deepseek_prompt = ChatPromptTemplate.from_template(deepseek_template)

# EXAONE 템플릿 생성
exaone_prompt = ChatPromptTemplate.from_template(exaone_template)



In [20]:
from langchain.prompts import ChatPromptTemplate

template = '''
<|system|>
You are a friendly chatbot specializing in artworks and general conversations.
Your primary role is to answer questions strictly based on the information provided in the document (context). 
If the requested information is not found in the document, respond with:
"The document does not contain this information." 

However, if the question is a general conversation or does not relate to the document, you should respond naturally as a conversational chatbot. 
You can talk about art history, artists, exhibitions, and general topics such as daily life, technology, and culture. 
Maintain a friendly and engaging tone, ensuring all responses are written in Korean.
Use **beautiful Markdown formatting** (headings, bullet points, bold or italic text) to enhance readability.
You must include artwork number.

<|context|>
{context}

<|user|>
Question: {question}

<|assistant|>
'''

# 프롬프트 템플릿 생성
prompt = ChatPromptTemplate.from_template(template)


In [34]:
retriever = faiss_db.as_retriever(
    search_kwargs={
        "k": 5,                # 검색 결과 개수
        "fetch_k": 15,         # 더 많은 결과 가져오기
        "mmr": True,           # MMR 활성화
        "mmr_beta": 0.3      # 다양성과 관련성 간 균형
    }
)


In [35]:
import re

class MarkdownOutputParser:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"</think>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [36]:
import re

class MarkdownOutputParser2:
    """Enhanced Markdown parser with additional formatting options."""

    def __call__(self, llm_output):
        """Extracts the assistant's response from after the </think> tag and formats it in Markdown."""
        if not llm_output or llm_output.strip() == "":
            return "❌ 모델에서 응답을 생성하지 못했습니다."

        # "</think>" 이후 텍스트 추출
        match = re.search(r"<\|assistant\|>\s*(.*)", llm_output, re.DOTALL)
        extracted_text = match.group(1).strip() if match else llm_output.strip()

        # Markdown 형식 적용
        formatted_output = f"""
### **🔹 모델 응답 결과**

{extracted_text}
"""
        return formatted_output.strip()  # 양 끝 공백 제거


In [24]:
from langchain.llms import HuggingFacePipeline
from transformers import pipeline

# 🔹 EXAONE 모델 로드
exaone_model_id = "LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct"
exaone_tokenizer = AutoTokenizer.from_pretrained(exaone_model_id)
exaone_model = AutoModelForCausalLM.from_pretrained(
    exaone_model_id,
    quantization_config=quantization_config,
    device_map="cuda",  # CUDA에서 자동 배치
    trust_remote_code=True
)


Loading checkpoint shards: 100%|██████████| 7/7 [00:25<00:00,  3.63s/it]


In [25]:
from transformers import pipeline

# 파이프라인 생성
exaone_pipe = pipeline(
    "text-generation",
    model=exaone_model,
    tokenizer=exaone_tokenizer,
    max_new_tokens=1024,  # 생성할 최대 토큰 수 증가
    do_sample=True,        # 샘플링 활성화
    temperature=0.1,      
    top_k=50,             
    repetition_penalty=1.05
)
# LangChain의 HuggingFacePipeline 사용
exaone_llm = HuggingFacePipeline(pipeline=exaone_pipe)

In [37]:
from langchain.schema.runnable import RunnableLambda

chain = (
    retriever
    | RunnableLambda(lambda docs: {  
        "context": "\n".join([doc.page_content for doc in docs]),  
        "question": query 
    })
    | deepseek_prompt
    | llm
    | MarkdownOutputParser()
    | (lambda x: {"reasoning": x})
    | exaone_prompt
    | exaone_llm
    | MarkdownOutputParser2()
)


In [47]:
query = "부산 전차 구조안전진단 및 보존처리에서 차량 구조는 어떻게 되었는가?"

In [48]:
response = chain.invoke({"question": query})
print(response)

KeyError: 11094

In [29]:
retrieved_docs = retriever.get_relevant_documents(query)
for i, doc in enumerate(retrieved_docs):
    print(f"Document {i+1}:")
    print(f"Content: {doc.page_content}")  # 문서의 실제 내용
    print(f"Metadata: {doc.metadata}")    # 메타데이터 (예: 출처, 페이지 등)
    print("-" * 50)


Document 1:
Content: {
    "title": "두 얼굴",
    "title_ch": "N/A",
    "title_eng": "Two Heads",
    "artist": "김정숙 a",
    "artist_eng": "KIM Chungsook a",
    "artwork_number": 4427,
    "year": "1990",
    "size": "115.5×38.5×38",
    "materials": "동",
    "category": "조각ㆍ설치",
    "description": "김정숙(1917-1991)은 1980년대 말부터 1990년대 초에 걸쳐 일련의 두상 조각들을 제작하였다. 이들 두상에는 대개 대리석 혹은 브론즈 등의 재료가 사용되었으며 조각에 맞추어 대좌가 함께 제작된 경우가 많았다. 같은 시기 <비상>(1976-1991) 연작에서처럼 두상의 형태는 극도로 단순화되어 있고 세부는 요약처리 되었으며, 부드럽게 정리된 표면과 정감어린 분위기 등을 특징으로 한다.<두 얼굴>(1990)은 원통형 대좌 위에 어른과 아이의 두상이 서로 밀착되어 있는 모습으로 조각되어 있다. 머리는 계란형이며 목은 기다랗게 과장되어 있다. 또한 얼굴은 이목구비 없이 극도로 단순화되어 있고, 표면은 매끈하게 다듬어져 있다. 이 작품은 버섯 혹은 식물의 싹과 같은 형태로써 유기적이고 생명감 넘치는 느낌을 자아낸다.",
    "read_count": 16
}
Metadata: {'title': '두 얼굴', 'artist': '김정숙 a', 'year': '1990', 'read_count': 16}
--------------------------------------------------
Document 2:
Content: {
    "title": "두 얼굴",
    "title_ch": "N/A",
    "title_eng": "Two Heads",
    "artist": "김정숙 a",
    "artist_eng

/tmp/ipykernel_798155/3695306690.py:1: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use :meth:`~invoke` instead.
  retrieved_docs = retriever.get_relevant_documents(query)


In [30]:
# 검색 수행: 유사도 점수와 함께 반환
docs_and_scores = retriever.vectorstore.similarity_search_with_score(query, k=5)

# 검색된 문서 수 출력
print(f"검색된 문서 수: {len(docs_and_scores)}")

# 각 문서의 파일명, 전체 내용, 유사도 점수 출력
for i, (doc, score) in enumerate(docs_and_scores, 1):
    print(f"\n문서 {i}:")
    print(f"  파일명: {doc.metadata.get('source', 'N/A')}")
    print(f"  유사도 점수: {score:.4f}")
    print(f"  전체 내용: {doc.page_content}")


검색된 문서 수: 5

문서 1:
  파일명: N/A
  유사도 점수: 0.8889
  전체 내용: {
    "title": "두 얼굴",
    "title_ch": "N/A",
    "title_eng": "Two Heads",
    "artist": "김정숙 a",
    "artist_eng": "KIM Chungsook a",
    "artwork_number": 4427,
    "year": "1990",
    "size": "115.5×38.5×38",
    "materials": "동",
    "category": "조각ㆍ설치",
    "description": "김정숙(1917-1991)은 1980년대 말부터 1990년대 초에 걸쳐 일련의 두상 조각들을 제작하였다. 이들 두상에는 대개 대리석 혹은 브론즈 등의 재료가 사용되었으며 조각에 맞추어 대좌가 함께 제작된 경우가 많았다. 같은 시기 <비상>(1976-1991) 연작에서처럼 두상의 형태는 극도로 단순화되어 있고 세부는 요약처리 되었으며, 부드럽게 정리된 표면과 정감어린 분위기 등을 특징으로 한다.<두 얼굴>(1990)은 원통형 대좌 위에 어른과 아이의 두상이 서로 밀착되어 있는 모습으로 조각되어 있다. 머리는 계란형이며 목은 기다랗게 과장되어 있다. 또한 얼굴은 이목구비 없이 극도로 단순화되어 있고, 표면은 매끈하게 다듬어져 있다. 이 작품은 버섯 혹은 식물의 싹과 같은 형태로써 유기적이고 생명감 넘치는 느낌을 자아낸다.",
    "read_count": 16
}

문서 2:
  파일명: N/A
  유사도 점수: 0.8956
  전체 내용: {
    "title": "두 얼굴",
    "title_ch": "N/A",
    "title_eng": "Two Heads",
    "artist": "김정숙 a",
    "artist_eng": "KIM Chungsook a",
    "artwork_number": 4379,
    "year": "1970",
    

### PDF DB에 추가

In [31]:
import faiss
import numpy as np
import pandas as pd
from sentence_transformers import SentenceTransformer
import fitz  # PyMuPDF
import pdfplumber
from langchain_core.documents import Document
from langchain.docstore.in_memory import InMemoryDocstore

# 1. 임베딩 모델 초기화
embedding_model = SentenceTransformer("nlpai-lab/KURE-v1")

# 2. FAISS 인덱스 로드 및 L2 거리 기반 인덱스 사용
def load_existing_faiss_db(index_path, embedding_dim):
    try:
        faiss_index = faiss.read_index(index_path)  # 기존 FAISS DB 로드
        print(f"FAISS index loaded from {index_path}.")
    except Exception as e:
        print(f"Error loading FAISS index from {index_path}. Error: {e}")
        # L2 거리 기반 인덱스 새로 생성 (embedding_dim: 벡터 차원)
        faiss_index = faiss.IndexFlatL2(embedding_dim)
        print("새로운 FAISS L2 인덱스 생성.")
    return faiss_index


# 3. PDF에서 텍스트와 표를 문단별로 추출
def extract_text_paragraphs(pdf_path):
    """
    PDF 문서에서 문단별로 텍스트 추출 (표는 제외)
    """
    doc_list = []
    pdf_doc = fitz.open(pdf_path)  # PyMuPDF로 PDF 열기

    with pdfplumber.open(pdf_path) as pdf:
        for page_num in range(len(pdf_doc)):
            page = pdf_doc[page_num]

            # 텍스트 추출 (문단 단위로 구분)
            text = page.get_text("text")
            paragraphs = text.split("\n\n")  # 빈 줄로 문단 구분

            # 표 추출
            tables = pdf.pages[page_num].extract_tables()
            table_texts = []
            if tables:
                for table in tables:
                    df = pd.DataFrame(table[1:], columns=table[0])  # 첫 행을 컬럼으로 설정
                    table_texts.append(df.to_string())
            else:
                table_texts = []  # 표가 없는 경우 빈 리스트

            # 문서 객체 생성 (문단별로 추가)
            for para in paragraphs:
                doc = Document(
                    page_content=para + "\n\n" + "\n\n".join(table_texts),  # 문단 + 표 포함
                    metadata={"page": page_num + 1}  # 페이지 정보 포함
                )
                doc_list.append(doc)

    return doc_list

# 4. 텍스트 벡터화 (SentenceTransformer 사용)
def text_to_vector(text):
    """
    텍스트를 벡터로 변환 (SentenceTransformer 사용)
    """
    vector = embedding_model.encode(text)  # 바로 numpy 배열 반환
    return vector


# 5. FAISS DB에 새로운 문서 추가 (기존 인덱스 ID와 겹치지 않게 추가)
def add_pdf_to_faiss(pdf_path, faiss_index, docstore, index_to_docstore_id):
    """
    주어진 PDF에서 문단을 추출하고 FAISS 인덱스에 추가 (진행 상황 출력)
    """
    # 기존 FAISS 인덱스에 추가된 벡터 수 파악 (새로운 벡터 ID가 기존과 겹치지 않도록 함)
    existing_vector_count = faiss_index.ntotal
    print(f"기존 FAISS 인덱스의 벡터 수: {existing_vector_count}")

    # PDF에서 문단별로 텍스트 추출
    documents = extract_text_paragraphs(pdf_path)
    
    # 문서의 총 문단 수
    total_paragraphs = len(documents)
    
    # 문단 벡터화
    vectors = []
    for idx, doc in enumerate(documents):
        # 진행 상황 출력
        progress = (idx + 1) / total_paragraphs * 100
        print(f"진행 상황: {progress:.2f}% - {idx+1}/{total_paragraphs} 문단 처리 중...")
        
        # 문단 벡터화
        vector = text_to_vector(doc.page_content)
        vectors.append(vector)

        # 문서 ID와 문서 내용 저장 (docstore)
        doc_id = existing_vector_count + len(vectors) - 1  # docstore에 새 문서의 ID는 마지막 인덱스 + 1
        docstore[doc_id] = doc
        index_to_docstore_id[existing_vector_count + idx] = str(doc_id)  # 벡터 ID와 문서 ID 연결

    # 벡터를 FAISS DB에 추가
    vectors = np.array(vectors).astype('float32')
    
    # 기존 FAISS 인덱스에 벡터 추가
    faiss_index.add(vectors)

    # FAISS DB 저장
    faiss.write_index(faiss_index, "./faiss_artworks_0303/index.faiss")
    print(f"FAISS index updated and saved to './faiss_artworks_0303/index.faiss'.")

# 6. 전체 실행 흐름
if __name__ == "__main__":
    # 벡터 차원 크기 (SentenceTransformer 모델에 맞는 차원)
    embedding_dim = 768  # 예시: KURE-v1 모델은 768차원

    # 기존 FAISS DB 로드 (기존 인덱스가 없으면 새로 생성)
    faiss_index = load_existing_faiss_db("./faiss_artworks_0303/index.faiss", embedding_dim)

    # 문서 저장소 초기화 (문서 ID와 내용 매핑)
    docstore = {}
    index_to_docstore_id = {i: i for i in range(len(docstore))}

    # 새로운 PDF 파일 경로 지정
    pdf_path = './PDF/dataset/artworks_10.pdf'

    # 새 PDF 문서를 기존 FAISS DB에 추가
    add_pdf_to_faiss(pdf_path, faiss_index, docstore, index_to_docstore_id)

    # 새로운 문서가 docstore에 잘 추가되었는지 확인
    print(f"현재 문서 저장소에 저장된 문서 개수: {len(docstore)}")
    print(f"첫 번째 문서 ID: {list(docstore.keys())[0]}")
    print(f"첫 번째 문서 내용: {docstore[list(docstore.keys())[0]].page_content[:100]}...")  # 첫 100글자만 출력


You try to use a model that was created with version 3.3.1, however, your version is 3.2.0. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





FAISS index loaded from ./faiss_artworks_0303/index.faiss.
기존 FAISS 인덱스의 벡터 수: 11029
진행 상황: 1.23% - 1/81 문단 처리 중...
진행 상황: 2.47% - 2/81 문단 처리 중...
진행 상황: 3.70% - 3/81 문단 처리 중...
진행 상황: 4.94% - 4/81 문단 처리 중...
진행 상황: 6.17% - 5/81 문단 처리 중...
진행 상황: 7.41% - 6/81 문단 처리 중...
진행 상황: 8.64% - 7/81 문단 처리 중...
진행 상황: 9.88% - 8/81 문단 처리 중...
진행 상황: 11.11% - 9/81 문단 처리 중...
진행 상황: 12.35% - 10/81 문단 처리 중...
진행 상황: 13.58% - 11/81 문단 처리 중...
진행 상황: 14.81% - 12/81 문단 처리 중...
진행 상황: 16.05% - 13/81 문단 처리 중...
진행 상황: 17.28% - 14/81 문단 처리 중...
진행 상황: 18.52% - 15/81 문단 처리 중...
진행 상황: 19.75% - 16/81 문단 처리 중...
진행 상황: 20.99% - 17/81 문단 처리 중...
진행 상황: 22.22% - 18/81 문단 처리 중...
진행 상황: 23.46% - 19/81 문단 처리 중...
진행 상황: 24.69% - 20/81 문단 처리 중...
진행 상황: 25.93% - 21/81 문단 처리 중...
진행 상황: 27.16% - 22/81 문단 처리 중...
진행 상황: 28.40% - 23/81 문단 처리 중...
진행 상황: 29.63% - 24/81 문단 처리 중...
진행 상황: 30.86% - 25/81 문단 처리 중...
진행 상황: 32.10% - 26/81 문단 처리 중...
진행 상황: 33.33% - 27/81 문단 처리 중...
진행 상황: 34.57% - 28/81 문단 처리 중...
진행 상황: 3

In [40]:
len(docstore)

81

In [41]:
print(f"문서 ID: {list(docstore.keys())[10]}")
print(f"첫 번째 문서 내용: {docstore[list(docstore.keys())[10]].page_content[:100]}...")

문서 ID: 11039
첫 번째 문서 내용: 미술작품의 보존 10
과학적 분석을 통한 유화 작품의 
하부 도상 제작 시기 검증연구 : 오지호 <풍경>(1927), <녹음>(1975)을 중심으로
19
18
4.3. 작품 주제 ...


In [ ]:
query = "부산 전차 구조안전진단 및 보존처리에서 차량 구조는 어떻게 되었는가?"

In [43]:
response = chain.invoke({"question": query})
print(response)

KeyError: 11094

In [49]:
# 매핑된 문서 ID 확인
for vector_id in range(faiss_index.ntotal):
    if vector_id in index_to_docstore_id:
        doc_id = index_to_docstore_id[vector_id]
        print(f"벡터 ID: {vector_id}, 문서 ID: {doc_id}, 문서 내용: {docstore[doc_id].page_content[:100]}...")
    else:
        print(f"벡터 ID: {vector_id}에 대한 문서 ID가 매핑되지 않았습니다.")


벡터 ID: 0에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 1에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 2에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 3에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 4에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 5에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 6에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 7에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 8에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 9에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 10에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 11에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 12에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 13에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 14에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 15에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 16에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 17에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 18에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 19에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 20에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 21에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 22에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 23에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 24에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 25에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 26에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 27에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 28에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 29에 대한 문서 ID가 매핑되지 않았습니다.
벡터 ID: 30에 대한 문서 ID가

KeyError: '11029'